In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
from technical.indicators import macd
from simulation.guru_tester import GuruTester

In [3]:
BUY = 1
SELL = -1
NONE = 0


def apply_signal(row):

    if row.direction == BUY and row.mid_l > row.EMA_100:
            return BUY
    if row.direction == SELL and row.mid_h < row.EMA_100:
            return SELL
    return NONE        

In [4]:
pair = "EUR_USD"
df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [6]:
df_an = macd(df_an)
df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

In [9]:
def apply_cross(row):
    if row.macd_delta > 0 and row.macd_delta_prev < 0:
        return BUY
    if row.macd_delta < 0 and row.macd_delta_prev > 0:
        return SELL
    return NONE 

In [12]:
df_an["macd_delta"] = df_an.MACD - df_an.SIGNAL
df_an["macd_delta_prev"] = df_an.macd_delta.shift(1)
df_an["direction"] = df_an.apply(apply_cross, axis=1)

In [15]:
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,MACD,SIGNAL,HIST,EMA_100,macd_delta,macd_delta_prev,direction
0,2015-06-05 03:00:00+00:00,539,1.12122,1.12144,1.12066,1.12105,1.12114,1.12136,1.12057,1.12097,1.12129,1.12151,1.12074,1.12113,-0.001146,-0.000266,-0.000880,1.118221,-0.000880,NaN,0
1,2015-06-05 04:00:00+00:00,479,1.12102,1.12244,1.12102,1.12180,1.12094,1.12235,1.12094,1.12172,1.12109,1.12254,1.12109,1.12188,-0.001138,-0.000440,-0.000697,1.118303,-0.000697,-0.000880,0
2,2015-06-05 05:00:00+00:00,703,1.12182,1.12243,1.12112,1.12205,1.12175,1.12236,1.12104,1.12187,1.12190,1.12250,1.12120,1.12223,-0.001098,-0.000572,-0.000526,1.118388,-0.000526,-0.000697,0
3,2015-06-05 06:00:00+00:00,4268,1.12202,1.12499,1.12180,1.12476,1.12185,1.12490,1.12172,1.12468,1.12220,1.12519,1.12188,1.12485,-0.000839,-0.000625,-0.000213,1.118532,-0.000213,-0.000526,0
4,2015-06-05 07:00:00+00:00,5720,1.12474,1.12800,1.12388,1.12748,1.12466,1.12793,1.12379,1.12741,1.12482,1.12809,1.12397,1.12754,-0.000409,-0.000582,0.000173,1.118735,0.000173,-0.000213,1


In [16]:
def run_pair(pair, time_d=1):
    df_an = pd.read_pickle(f"../data/{pair}_H{time_d}.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an = macd(df_an)
    df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
    df_an["macd_delta"] = df_an.MACD - df_an.SIGNAL
    df_an["macd_delta_prev"] = df_an.macd_delta.shift(1)
    df_an["direction"] = df_an.apply(apply_cross, axis=1)
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
             'direction', 'EMA_100']
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True,
        time_d = time_d
    )
    
    gt.run_test()
    return gt.df_results
    

In [17]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p, time_d=1)))

prepare_data...
run_test...
Result: -827.5
prepare_data...
run_test...
Result: -106.0
prepare_data...
run_test...
Result: -27.0
prepare_data...
run_test...
Result: -229.0


In [18]:
for r in res:
    print(r['pair'], r['res'].result.sum())

AUD_NZD -827.5
EUR_USD -106.0
GBP_JPY -27.0
USD_CAD -229.0
